In [ ]:
from multiprocessing import Queue
from dataclasses import dataclass
from multiprocessing import Queue, Process
from multiprocessing.queues import Empty
import json
from time import sleep
import sys
import pdb


monitor_events_queue = Queue()
@dataclass
class Event:
    source: str       # отправитель
    destination: str  # получатель
    operation: str    # чего хочет (запрашиваемое действие)
    parameters: str   # с какими параметрами


# формат управляющих команд для монитора
@dataclass
class ControlEvent:
    operation: str

# список разрешенных сочетаний сигналов светофора
# любые сочетания, отсутствующие в этом списке, запрещены
traffic_lights_allowed_configurations = [
    {"direction_1": "red", "direction_2": "green"},
    {"direction_1": "red", "direction_2": "red"},
    {"direction_1": "red", "direction_2": "yellow"},
    {"direction_1": "yellow", "direction_2": "yellow"},
    {"direction_1": "off", "direction_2": "off"},
    {"direction_1": "green", "direction_2": "red"},
    {"direction_1": "green", "direction_2": "yellow"},

    {"direction_1": "red", "direction_2": "yellow_blinking"},
    {"direction_1": "yellow_blinking", "direction_2": "yellow_blinking"},
    {"direction_1": "green", "direction_2": "yellow_blinking"},
    {"direction_1": "yellow_blinking", "direction_2": "green"},
    {"direction_1": "yellow_blinking", "direction_2": "red"},
    {"direction_1": "yellow", "direction_2": "yellow_blinking"},
]


# Класс, реализующий поведение монитора безопасности
class Monitor(Process):

    def __init__(self, events_q: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        self._events_q = events_q  # очередь событий для монитора (входящие сообщения)
        self._control_q = Queue()  # очередь управляющих команд (например, для остановки монитора)
        self._entity_queues = {}   # словарь очередей известных монитору сущностей
        self._force_quit = False   # флаг завершения работы монитора

    # регистрация очереди новой сущности
    def add_entity_queue(self, entity_id: str, queue: Queue):
        print(f"[монитор] регистрируем сущность {entity_id}")
        self._entity_queues[entity_id] = queue

    def _check_mode(self, mode_str: str) -> bool:
        mode_ok = False
        try:
            # извлечём структуру из строки, в случае ошибки запретим изменение режима
            mode = json.loads(mode_str)
            if mode.get("direction_1") == "yellow_blinking" or mode.get("direction_2") == "yellow_blinking":
                return True
            return mode in traffic_lights_allowed_configurations
        except json.JSONDecodeError:
            print(f"[монитор] Invalid JSON in mode string: {mode_str}", file=sys.stderr)
            return False
        except Exception as e:
            print(f"[монитор] Error checking mode: {e}", file=sys.stderr)
            return False

            # проверим входит ли запрашиваемый режим в список разрешённых
            print(f"[монитор] проверяем конфигурацию {mode}")
            if mode in traffic_lights_allowed_configurations:
                # такой режим найден, можно активировать
                mode_ok = True
        except:
            mode_ok = False
        return mode_ok

    # проверка политик безопасности
    def _check_policies(self, event):
        print(f'[монитор] обрабатываем событие {event}')

        # default deny: всё, что не разрешено, запрещено по умолчанию!
        authorized = False

        # проверка на входе, что это экземпляр класса Event,
        # т.е. имеет ожидаемый формат
        if not isinstance(event, Event):
            print(f"[монитор] событие не разрешено политиками безопасности: {event}", file=sys.stderr)
            return False

        #
        #  политики безопасности
        #

        # пример политики безопасности
        if event.source == "ControlSystem" \
                and event.destination == "LightsGPIO" \
                and event.operation == "set_mode" \
                and self._check_mode(event.parameters):
            authorized = True

        if authorized is False:
            print("[монитор] событие не разрешено политиками безопасности")
        return authorized

    # выполнение разрешённого запроса
    # метод должен вызываться только после проверки политик безопасности
    def _proceed(self, event):
        print(f'[монитор] отправляем запрос {event}')
        try:
            # найдём очередь получателя события
            dst_q: Queue = self._entity_queues[event.destination]
            # и положим запрос в эту очередь
            dst_q.put(event)
        except  Exception as e:
            # например, запрос пришёл от или для неизвестной сущности
            print(f"[монитор] ошибка выполнения запроса {e}")

    # основной код работы монитора безопасности
    def run(self):
        print('[монитор] старт')

        # в цикле проверяет наличие новых событий,
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим,
                # чтобы можно было завершить работу монитора,
                # не дожидаясь нового сообщения
                event = self._events_q.get_nowait()
                # сюда попадаем только в случае получение события,
                # теперь нужно проверить политики безопасности
                authorized = self._check_policies(event)
                if authorized:
                    # если политиками запрос авторизован - выполняем
                    self._proceed(event)
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[монитор] ошибка обработки {e}, {event}")
            self._check_control_q()
        print('[монитор] завершение работы')

    # запрос на остановку работы монитора безопасности для завершения работы
    # может вызываться вне процесса монитора
    def stop(self):
        # поскольку монитор работает в отдельном процессе,
        # запрос помещается в очередь, которая проверяется из процесса монитора
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[монитор] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass



class ControlSystem(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()

    # выдаёт собственную очередь для взаимодействия
    def entity_queue(self):
        return self._own_queue

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        print(f'[{self.__class__.__name__}] отправляем тестовый запрос')

        mode = {"direction_1": "green", "direction_2": "yellow_blinking"}
        # запрос для сущности WorkerB - "скажи hello"
        event = Event(source="ControlSystem",
                      destination="LightsGPIO",
                      operation="set_mode",
                      parameters=json.dumps(mode))
        monitor_events_queue.put(event)

        self.monitor_queue.put(event)
        print(f'[{self.__class__.__name__}] завершение работы')


class LightsGPIO(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        self._current_mode = {"direction_1": "off", "direction_2": "off"}  # Начальное состояние
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self._blinking_yellow = False  # Флаг, указывающий, активен ли режим моргающего желтого
        self._blinking_state = False  # Изначально выключено

    def entity_queue(self):
        return self._own_queue

    # основной код сущности
    def run(self):
        """
        Основной цикл процесса LightsGPIO, обрабатывающий события из своей очереди.

        Обрабатывает события для установки режима светофора и обрабатывает "yellow_blinking"
        специальным образом для имитации эффекта мигания.
        """
        print(f'[{self.__class__.__name__}] старт')
        while True:
            try:
                # Сначала проверяем наличие входящих событий
                try:
                    event: Event = self._own_queue.get_nowait()  # Неблокирующее получение
                    if event.operation == "set_mode":
                        try:
                            mode = json.loads(event.parameters)

                            if mode == {"direction_1": "yellow_blinking", "direction_2": "yellow_blinking"}:
                                print(f"[{self.__class__.__name__}] Активирован режим моргающего желтого")
                                self._blinking_yellow = True
                            elif mode == {"direction_1": "no_yellow_blinking", "direction_2": "no_yellow_blinking"}:
                                print(f"[{self.__class__.__name__}] Деактивирован режим моргающего желтого")
                                self._blinking_yellow = False
                            else:
                                self._current_mode = mode
                                print(f"[{self.__class__.__name__}] {event.source} запрашивает изменение режима {event.parameters}")
                                print(f"[{self.__class__.__name__}] новый режим: {self._current_mode}!")


                        except (json.JSONDecodeError, TypeError) as e:
                            print(f"[{self.__class__.__name__}] Ошибка при декодировании JSON: {e}")

                except Empty:
                    pass  # В данный момент нет событий

                # Комбинируем основной цвет с моргающим желтым, если он активен
                output_mode = self._current_mode.copy()  # Создаем копию, чтобы не изменять оригинал

                if self._blinking_yellow:
                    self._blinking_state = not self._blinking_state  # Переключаем состояние мигания

                    # Накладываем эффект мигания
                    for direction in output_mode:
                        if output_mode[direction] != "off":
                            output_mode[direction] = "yellow" if self._blinking_state else output_mode[direction]

                # Отображаем комбинированный режим
                print(f"[{self.__class__.__name__}] Текущий режим: {output_mode}")
                sleep(0.5)



            except Exception as e:
                print(f"[{self.__class__.__name__}] Общая ошибка: {e}")
                break  # Выходим при неожиданных ошибках

        print(f'[{self.__class__.__name__}] завершение работы')


# Пример использования
if __name__ == '__main__':
    from multiprocessing import Process, Queue
    from dataclasses import dataclass
    from time import sleep

    @dataclass
    class Event:
        source: str
        destination: str
        operation: str
        parameters: str

    # Создаем очередь для связи с монитором безопасности (фиктивный)
    monitor_queue = Queue()

    # Создаем процесс LightsGPIO
    lights = LightsGPIO(monitor_queue)

    # Получаем очередь сущности для отправки событий
    lights_queue = lights.entity_queue()

    # Запускаем процесс LightsGPIO
    lights.start()

    # Устанавливаем красный и зеленый
    event_red_green = Event(source="ControlSystem",
                                destination="LightsGPIO",
                                operation="set_mode",
                                parameters='{"direction_1": "red", "direction_2": "green"}')
    lights_queue.put(event_red_green)
    sleep(2)

    # Активируем режим моргающего желтого
    event_blinking_yellow = Event(source="ControlSystem",
                                      destination="LightsGPIO",
                                      operation="set_mode",
                                      parameters='{"direction_1": "yellow_blinking", "direction_2": "yellow_blinking"}')
    lights_queue.put(event_blinking_yellow)
    sleep(5)  # Смотрим на красный/зеленый с моргающим желтым

    # Отключаем моргающий жёлтый
    event_no_blinking_yellow = Event(source="ControlSystem",
                                      destination="LightsGPIO",
                                      operation="set_mode",
                                      parameters='{"direction_1": "no_yellow_blinking", "direction_2": "no_yellow_blinking"}')
    lights_queue.put(event_no_blinking_yellow)
    sleep(2)

    # Ставим зеленый и красный
    event_green_red = Event(source="ControlSystem",
                                destination="LightsGPIO",
                                operation="set_mode",
                                parameters='{"direction_1": "green", "direction_2": "red"}')
    lights_queue.put(event_green_red)
    sleep(2)


    lights.terminate()
    lights.join()

    monitor_queue.close()
    lights_queue.close()

[LightsGPIO] старт
[LightsGPIO] ControlSystem запрашивает изменение режима {"direction_1": "red", "direction_2": "green"}
[LightsGPIO] новый режим: {'direction_1': 'red', 'direction_2': 'green'}!
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': 'green'}
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': 'green'}
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': 'green'}
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': 'green'}
[LightsGPIO] Активирован режим моргающего желтого
[LightsGPIO] Текущий режим: {'direction_1': 'yellow', 'direction_2': 'yellow'}
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': 'green'}
[LightsGPIO] Текущий режим: {'direction_1': 'yellow', 'direction_2': 'yellow'}
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': 'green'}
[LightsGPIO] Текущий режим: {'direction_1': 'yellow', 'direction_2': 'yellow'}
[LightsGPIO] Текущий режим: {'direction_1': 'red', 'direction_2': '